In [1]:
#import liabraries
import string
import re
import codecs
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn import feature_extraction
from sklearn import linear_model
from sklearn import pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [21]:
eng_df = pd.read_csv("Dataset/eng_news_2015_10K-sentences.txt","utf-8", header=None, names=["English"])
ger_df = pd.read_csv("Dataset/deu_mixed-typical_2011_10K-sentences.txt","utf-8", header=None, names=["German"])
fre_df = pd.read_csv("Dataset/fra_mixed_2009_10K-sentences.txt","utf-8", header=None, names=["French"])
spa_df = pd.read_csv("Dataset/spa_news_2011_10K-sentences.txt","utf-8", header=None, names=["Spanish"])

<ipython-input-21-70216dc1dc7c>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  eng_df = pd.read_csv("Dataset/eng_news_2015_10K-sentences.txt","utf-8", header=None, names=["English"])
<ipython-input-21-70216dc1dc7c>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ger_df = pd.read_csv("Dataset/deu_mixed-typical_2011_10K-sentences.txt","utf-8", header=None, names=["German"])
<ipython-input-21-70216dc1dc7c>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid th

In [3]:
#get punctuation
for char in string.punctuation:
    print(char, end=' ')
translate_table = dict((ord(char), None) for char in string.punctuation)

! " # $ % & ' ( ) * + , - . / : ; < = > ? @ [ \ ] ^ _ ` { | } ~ 

In [4]:
data_eng = []
lang_eng = []
for i,line in eng_df.iterrows():
    line = line['English']
    if len(line) != 0:
        line = line.lower()
        line = re.sub(r"\d+","",line)
        line = line.translate(translate_table)
        data_eng.append(line)
        lang_eng.append("English")

In [5]:
data_ger = []
lang_ger = []
for i,line in ger_df.iterrows():
    line = line['German']
    if len(line) != 0:
        line = line.lower()
        line = re.sub(r"\d+","",line)
        line = line.translate(translate_table)
        data_ger.append(line)
        lang_ger.append("German")

In [6]:
data_fre = []
lang_fre = []
for i,line in fre_df.iterrows():
    line = line['French']
    if len(line) != 0:
        line = line.lower()
        line = re.sub(r"\d+","",line)
        line = line.translate(translate_table)
        data_fre.append(line)
        lang_fre.append("French")

In [7]:
data_spa = []
lang_spa = []
for i,line in spa_df.iterrows():
    line = line['Spanish']
    if len(line) != 0:
        line = line.lower()
        line = re.sub(r"\d+","",line)
        line = line.translate(translate_table)
        data_spa.append(line)
        lang_spa.append("Spanish")

In [23]:
df = pd.DataFrame({"Text":data_eng+data_ger+data_fre+data_spa,"Language":lang_eng+lang_ger+lang_fre+lang_spa})
df.shape

(39958, 2)

In [24]:
X, y = df.iloc[:,0],df.iloc[:,1]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [25]:
# vectorizer and model fitting pipeline
vectorizer = feature_extraction.text.TfidfVectorizer(ngram_range=(1,3), analyzer='char')
pipe_lr_r13 = pipeline.Pipeline([
    ('vectorizer', vectorizer),
    ('clf', linear_model.LogisticRegression())
])
# model fitting
pipe_lr_r13.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(analyzer='char', ngram_range=(1, 3))),
                ('clf', LogisticRegression())])

In [26]:
#model prediction
y_predict = pipe_lr_r13.predict(X_test)
#model evaluation
acc = (metrics.accuracy_score(y_test, y_predict))*100
print("Accuracy: ",acc,"%")


Accuracy:  99.76226226226225 %


In [27]:
matrix = metrics.confusion_matrix(y_test, y_predict)
print("Confusion Matrix: \n", matrix)


Confusion Matrix: 
 [[2000    0    5    1]
 [   0 1947    5    1]
 [   2    1 1994    0]
 [   1    1    2 2032]]


In [28]:
#save the model
lrFile = open("LRModel.pck1", "wb")
pickle.dump(pipe_lr_r13, lrFile)
lrFile.close()